In [1]:
! gdown 1yEjicXasMGFaALQobQWgsSxQgLEtxruC
! unzip /content/ThaiDepression.zip

Downloading...
From: https://drive.google.com/uc?id=1yEjicXasMGFaALQobQWgsSxQgLEtxruC
To: /content/ThaiDepression.zip
100% 41.1k/41.1k [00:00<00:00, 45.8MB/s]
Archive:  /content/ThaiDepression.zip
  inflating: ThaiDepression/AHB_01.csv  
  inflating: ThaiDepression/AHB_02.csv  
  inflating: ThaiDepression/AHB_03.csv  
  inflating: ThaiDepression/AHB_04.csv  
  inflating: ThaiDepression/AHB_05.csv  
  inflating: ThaiDepression/AHB_06.csv  
  inflating: ThaiDepression/AHB_07.csv  
  inflating: ThaiDepression/AHB_08.csv  
  inflating: ThaiDepression/AHB_09.csv  
  inflating: ThaiDepression/AHB_10.csv  
  inflating: ThaiDepression/AHB_11.csv  
  inflating: ThaiDepression/AHB_12.csv  
  inflating: ThaiDepression/AHB_13.csv  
  inflating: ThaiDepression/All_AHB.csv  


In [2]:
import pandas as pd
import numpy as np
import os
from numpy import array
from math import sqrt
from numpy import mean
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from keras.preprocessing import sequence
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GRU, BatchNormalization
from pandas import read_csv
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, SimpleRNN
import json
import psutil 
from datetime import datetime, timezone, timedelta

In [3]:
file_list = os.listdir('ThaiDepression')
file_list.sort()
df_list = []
for i in range(14):
    df_list.append(pd.read_csv('ThaiDepression/' + file_list[i], index_col=0))

In [4]:
df_list

[         Chiang_Rai    Nan  Phayao  Phrae  Chiang_Mai  Mae_Hong_Son  Lampang  \
 2016-10       10383   5315    4266   4057       15408          2002     9011   
 2016-11       10383   5315    4265   4057       15406          2002     9011   
 2016-12       10382   5315    4265   4057       15404          2002     9011   
 2017-01       10382   5655    4265   4057       15393          2002     9011   
 2017-02       10382   5655    4264   4057       15392          2002     9011   
 ...             ...    ...     ...    ...         ...           ...      ...   
 2022-03       25223   9711   10788   7988       47292          6924    17444   
 2022-04       25456   9781   10932   8095       48020          7004    17606   
 2022-05       25619   9833   10977   8122       48155          7026    17703   
 2022-06       25836   9884   11069   8161       48616          7090    17783   
 2022-07       26174  10019   11163   8250       49096          7169    17951   
 
          Lamphun   AHB_1 

In [5]:
# split a univariate sequence into samples
def uni_split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


    # split a multivariate sequence into samples
def multi_split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


def get_scaler(scaler):
    scalers = {
        "minmax": MinMaxScaler,
        "standard": StandardScaler,
        "maxabs": MaxAbsScaler,
        "robust": RobustScaler,
    }
    return scalers.get(scaler.lower())()

In [6]:
%matplotlib inline
# define scope of configs
AHB_list = [11, 12, 13]
GRU_hidden_units = [8, 16, 32, 64, 128]
n_layer = [1, 2, 3]
n_batch = [2, 4, 8, 16, 32, 64, 128]
n_steps = [5]
patience_list = [3, 5, 7, 10]
mul_uni = ['uni','mul']
dropouts = [0.2, 0.3, 0.5]


timezone_offset = 7.0  # Bangkok Time (UTC+07:00)
tzinfo = timezone(timedelta(hours=timezone_offset))
count = 1
performance_json = []
for g in AHB_list:
    for h in patience_list:
        for a in n_steps:
            for b in dropouts:
                for c in GRU_hidden_units:
                    for d in n_layer:
                        for e in n_batch:
                            for f in mul_uni:

                                print('PROGRESS: [ ' + str(count) + ' / ' + str(len(dropouts)*len(AHB_list)*len(n_steps)*len(GRU_hidden_units)*len(n_layer)*len(n_batch)*len(patience_list)*len(mul_uni)) + ' ]')
                                
                                if f == 'uni':
                                    if g == 13:
                                        raw_seq = list(df_list[g]['Country_Level'])
                                        ahb = 'Country_Level'
                                    else:    
                                        raw_seq = list(df_list[g]['AHB_'+str(g+1)])
                                        ahb = 'AHB_'+str(g+1)
                                    n_features = 1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq).reshape(len(raw_seq),-1))
                                    X, y = uni_split_sequence(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))
                                if f == 'mul':
                                    if g == 13:
                                        raw_seq = list(df_list[13].drop(df_list[13].columns[:77], axis=1).values)
                                        ahb = 'Country_Level'
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[13]['Country_Level']).reshape(len(df_list[13]['Country_Level']),-1))
                                    else:    
                                        raw_seq = list(df_list[g].drop(['Country_Level'], axis=1).values)
                                        ahb = 'AHB_'+str(g+1)
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[g]['AHB_'+str(g+1)]).reshape(len(df_list[13]['AHB_'+str(g+1)]),-1))
                                    n_features = len(raw_seq[0])-1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq))
                                    X, y = multi_split_sequences(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))

                                callback = EarlyStopping(monitor='loss', patience=h)
                                model = Sequential()
                                if d == 1:
                                    model.add(GRU(units=c, return_sequences=False, input_shape=(a,n_features), activation='tanh'))
                                    model.add(Dropout(b))
                                if d == 2:
                                    model.add(GRU(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh'))
                                    model.add(Dropout(b))
                                    model.add(GRU(units=c, return_sequences=False, activation='tanh'))
                                    model.add(Dropout(b))
                                if d == 3:
                                    model.add(GRU(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh'))
                                    model.add(Dropout(b))
                                    model.add(GRU(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh'))
                                    model.add(Dropout(b))
                                    model.add(GRU(units=c, return_sequences=False, activation='tanh'))
                                    model.add(Dropout(b))
                                model.add(Dense(units=1, kernel_initializer='he_uniform', activation='linear'))
                                model.compile(loss='mse', optimizer='adam')  
                                history = model.fit(X_train, y_train, epochs=2000, batch_size=e, verbose=0, callbacks=[callback])
                                last_epoch = len(history.history['loss']) 

                                yhat = model.predict(X_test, verbose=0)
                                yhat = yhat.reshape(yhat.shape[0],-1)
                                mae = mean_absolute_error(y_test, yhat)
                                mse = mean_squared_error(y_test, yhat)
                                rmse = mean_squared_error(y_test, yhat, squared = False)
                                mape = mean_absolute_percentage_error(y_test, yhat)
                                
                                if f == 'uni':
                                    yhat_inverse = scaler.inverse_transform(yhat)
                                    y_test_inverse = scaler.inverse_transform(y_test)
                                if f == 'mul':
                                    yhat_inverse = scaler_y.inverse_transform(yhat)
                                    y_test_inverse = scaler_y.inverse_transform(y_test.reshape(len(y_test),-1))


                                mae_inverse = mean_absolute_error(y_test_inverse, yhat_inverse)
                                mse_inverse = mean_squared_error(y_test_inverse, yhat_inverse)
                                rmse_inverse = mean_squared_error(y_test_inverse, yhat_inverse, squared = False)
                                mape_inverse = mean_absolute_percentage_error(y_test_inverse, yhat_inverse)

                                performance_json.append({'model': 'GRU', 'AHB_number': ahb, 'Multi_Uni': f, \
                                                         'n_layer': d, 'GRU_hidden_units': c, 'dropouts': b,'n_batch': e,\
                                                         'n_steps': a, 'last_epoch': last_epoch, 'patience_list': h, 'MAE_normalize': float(mae), \
                                                         'MSE_normalize': float(mse), 'RMSE_normalize': float(rmse), 'MAPE_normalize': float(mape), \
                                                         'MAE': float(mae_inverse), 'MSE': float(mse_inverse), 'RMSE': float(rmse_inverse), 'MAPE': float(mape_inverse)})
                                

                                json_formatted_str = json.dumps(performance_json[-1], indent=2)
                                print(json_formatted_str)
                                print('Percent CPU Usage: ' + str(psutil.cpu_percent()))
                                print('Percent Ram Usage: ' + str(psutil.virtual_memory()[2]))
                                print(datetime.now(tzinfo))

                                count = count + 1

PROGRESS: [ 1 / 7560 ]
{
  "model": "GRU",
  "AHB_number": "AHB_12",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "GRU_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 2,
  "n_steps": 5,
  "last_epoch": 12,
  "patience_list": 3,
  "MAE_normalize": 0.16424726516056007,
  "MSE_normalize": 0.02713105468137349,
  "RMSE_normalize": 0.1647150712028911,
  "MAPE_normalize": 0.17260819884776493,
  "MAE": 7061.481026785715,
  "MSE": 50148964.468340196,
  "RMSE": 7081.593356606986,
  "MAPE": 0.07547127234695963
}
Percent CPU Usage: 42.0
Percent Ram Usage: 9.2
2022-10-14 00:42:35.991551+07:00
PROGRESS: [ 2 / 7560 ]
{
  "model": "GRU",
  "AHB_number": "AHB_12",
  "Multi_Uni": "mul",
  "n_layer": 1,
  "GRU_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 2,
  "n_steps": 5,
  "last_epoch": 17,
  "patience_list": 3,
  "MAE_normalize": 0.04865813182980621,
  "MSE_normalize": 0.0024034995742340357,
  "RMSE_normalize": 0.049025499224730346,
  "MAPE_normalize": 0.051055193560424525,
  "MAE": 2091.95870535714

{
  "model": "GRU",
  "AHB_number": "AHB_12",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "GRU_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 8,
  "n_steps": 5,
  "last_epoch": 14,
  "patience_list": 3,
  "MAE_normalize": 0.1931424644594187,
  "MSE_normalize": 0.03755778746218359,
  "RMSE_normalize": 0.19379831645858947,
  "MAPE_normalize": 0.20292447585757917,
  "MAE": 8303.77232142857,
  "MSE": 69421712.46557617,
  "RMSE": 8331.969302966507,
  "MAPE": 0.08873904939457522
}
Percent CPU Usage: 53.7
Percent Ram Usage: 11.7
2022-10-14 00:43:02.200491+07:00
PROGRESS: [ 6 / 7560 ]


{
  "model": "GRU",
  "AHB_number": "AHB_12",
  "Multi_Uni": "mul",
  "n_layer": 1,
  "GRU_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 8,
  "n_steps": 5,
  "last_epoch": 10,
  "patience_list": 3,
  "MAE_normalize": 0.1698480411517274,
  "MSE_normalize": 0.029387145499277848,
  "RMSE_normalize": 0.1714267934112922,
  "MAPE_normalize": 0.17812442105777176,
  "MAE": 7302.274553571428,
  "MSE": 54319111.12248884,
  "RMSE": 7370.150006783366,
  "MAPE": 0.07797437220856088
}
Percent CPU Usage: 63.1
Percent Ram Usage: 11.8
2022-10-14 00:43:05.827713+07:00
PROGRESS: [ 7 / 7560 ]
{
  "model": "GRU",
  "AHB_number": "AHB_12",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "GRU_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 16,
  "n_steps": 5,
  "last_epoch": 11,
  "patience_list": 3,
  "MAE_normalize": 0.24024770017228755,
  "MSE_normalize": 0.05811088170142669,
  "RMSE_normalize": 0.24106198725934933,
  "MAPE_normalize": 0.2524141496019566,
  "MAE": 10328.96986607143,
  "MSE": 107412051.25623

KeyboardInterrupt: ignored

In [ ]:
name = "all_performance_GRU_5nsteps_AHB12-13-C"
json.dump(performance_json, open(name + ".json","w"))
df_json = pd.read_json('/content/' + name + '.json')
df_json.to_csv('/content/' + name + '.csv', index=False)
#! cp /content/all_performance_GRU_5nsteps_AHB12-13-C.json /content/drive/MyDrive/Depression
! cp /content/all_performance_GRU_5nsteps_AHB12-13-C.csv /content/drive/MyDrive/Depression

In [ ]:
df_json

,model,AHB_number,Multi_Uni,n_layer,GRU_hidden_units,dropouts,n_batch,n_steps,last_epoch,patience_list,MAE_normalize,MSE_normalize,RMSE_normalize,MAPE_normalize,MAE,MSE,RMSE,MAPE
0,GRU,AHB_12,uni,1,8,0.2,2,5,16,3,0.035619,0.001312,0.036222,0.037308,1531.378348,2.425171e+06,1557.296032,0.016343
1,GRU,AHB_12,mul,1,8,0.2,2,5,25,3,0.072718,0.005489,0.074088,0.076114,3126.373884,1.014604e+07,3185.284680,0.033356
2,GRU,AHB_12,uni,1,8,0.2,4,5,15,3,0.101252,0.010391,0.101937,0.106263,4353.140625,1.920700e+07,4382.578975,0.046498
3,GRU,AHB_12,mul,1,8,0.2,4,5,27,3,0.063064,0.004031,0.063488,0.066189,2711.309152,7.450408e+06,2729.543488,0.028962
4,GRU,AHB_12,uni,1,8,0.2,8,5,21,3,0.232467,0.054429,0.233300,0.244220,9994.454241,1.006060e+08,10030.252823,0.106803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7555,GRU,Country_Level,mul,3,128,0.5,32,5,39,10,0.020257,0.000436,0.020879,0.021013,13025.250000,1.802366e+08,13425.221591,0.010612
7556,GRU,Country_Level,uni,3,128,0.5,64,5,62,10,0.042779,0.001850,0.043015,0.044550,27506.500000,7.649677e+08,27658.049279,0.022455
7557,GRU,Country_Level,mul,3,128,0.5,64,5,41,10,0.082230,0.006807,0.082504,0.085681,52872.910714,2.814253e+09,53049.531190,0.043175
7558,GRU,Country_Level,uni,3,128,0.5,128,5,31,10,0.125343,0.015744,0.125475,0.130735,80594.196429,6.509186e+09,80679.526441,0.065844
